In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange
import rosbags
import json
import open3d as o3d
import pandas as pd


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
file_path = 'datasets/json/box_plant1/box_plant1_frame300_400.json'
# Open and read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Now 'data' is a dictionary
num_key = len(data.keys())
if num_key > 0:
    print(f"Data loaded, number of frames = {num_key}")

Data loaded, number of frames = 100


In [4]:
frame = 350
points = np.array(data[str(frame)])

In [5]:
# Convert numpy array to open3d point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])

In [24]:
points = np.array([[5,5,5],[2,2,2],[-1,-10,-6]])
threshold = 5
distance = (points[:,0]**2 + points[:,1]**2 + points[:,2]**2)**0.5
mask = distance < threshold
x = shortPassFilter(points, threshold)
print(x)


[]


In [32]:
from preprocess import shortPassFilter

In [33]:
points = np.array([[5,5,5],[2,2,2],[-1,-10,-6]])
threshold = 5
distance = (points[:,0]**2 + points[:,1]**2 + points[:,2]**2)**0.5
mask = distance < threshold
x = shortPassFilter(points, threshold)
print(x)

[]
